# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from experiments import *
#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=3)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [2]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = d
c1 = 0.9
c2 = 0.9

#prep error and full POVM error
spam_target = SPAM(d=d)

init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

spam_target = SPAM(d=d,
                  init = init_target,
                  povm = povm_target)

kraus_target = KrausMap(
                        U=generate_unitary(d=d), 
                        c=0.5, 
                        d=d, 
                        rank=rank-1,
                        spam = spam_target
                        )

### Generate Synthetic Data with Noise

In [3]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

#add noise
targets_spam = add_noise_to_probs(targets_spam, 0.01)

inputs_map, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = 1800, 
                                        trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_map(state)
targets_map = measurement(state, U_basis, spam_target.povm)

#add noise
targets_map = add_noise_to_probs(targets_map, 0.0316)

### Fit Model, Initial and Corruption Matrix

In [4]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 300,
                    verbose = False,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 N = N_spam,
                 verbose = False,
                )


kraus_model = KrausMap(
                       d = d, 
                       rank = d**2,
                       spam = spam_model,
                       )

  0%|          | 0/300 [00:00<?, ?it/s]

4.0923774930680634e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0.0010705312804176836


In [5]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model = ModelQuantumMap(
                        q_map = kraus_model,
                        loss = ProbabilityLoss(0.001),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        )

model.train(inputs = inputs_map,
            targets = targets_map,
            #inputs_val = inputs_map,
            targets_val = [kraus_target],
            num_iter = 2000,
            N = 500,
            verbose = True,
            )

  0%|          | 0/2000 [00:00<?, ?it/s]

Step:0, train: 0.19297, val: 0.10299
Step:1, train: 0.19550, val: 0.10499
Step:2, train: 0.18248, val: 0.10705
Step:3, train: 0.18915, val: 0.10922
Step:4, train: 0.18195, val: 0.11142
Step:5, train: 0.18558, val: 0.11371
Step:6, train: 0.18197, val: 0.11608
Step:7, train: 0.17765, val: 0.11847
Step:8, train: 0.17641, val: 0.12091
Step:9, train: 0.17958, val: 0.12338
Step:10, train: 0.17217, val: 0.12589
Step:11, train: 0.17301, val: 0.12845
Step:12, train: 0.17235, val: 0.13106
Step:13, train: 0.16876, val: 0.13374
Step:14, train: 0.16620, val: 0.13646
Step:15, train: 0.16122, val: 0.13928
Step:16, train: 0.15923, val: 0.14214
Step:17, train: 0.16079, val: 0.14505
Step:18, train: 0.15692, val: 0.14803
Step:19, train: 0.15133, val: 0.15104
Step:20, train: 0.15568, val: 0.15412
Step:21, train: 0.14905, val: 0.15726
Step:22, train: 0.14629, val: 0.16048
Step:23, train: 0.14305, val: 0.16376
Step:24, train: 0.14179, val: 0.16711
Step:25, train: 0.13898, val: 0.17050
Step:26, train: 0.1385

Step:213, train: 0.01317, val: 0.68076
Step:214, train: 0.01376, val: 0.68193
Step:215, train: 0.01397, val: 0.68311
Step:216, train: 0.01339, val: 0.68428
Step:217, train: 0.01358, val: 0.68544
Step:218, train: 0.01314, val: 0.68660
Step:219, train: 0.01351, val: 0.68775
Step:220, train: 0.01321, val: 0.68888
Step:221, train: 0.01334, val: 0.69001
Step:222, train: 0.01335, val: 0.69113
Step:223, train: 0.01336, val: 0.69225
Step:224, train: 0.01361, val: 0.69336
Step:225, train: 0.01311, val: 0.69447
Step:226, train: 0.01307, val: 0.69558
Step:227, train: 0.01268, val: 0.69668
Step:228, train: 0.01321, val: 0.69779
Step:229, train: 0.01289, val: 0.69888
Step:230, train: 0.01272, val: 0.69997
Step:231, train: 0.01272, val: 0.70105
Step:232, train: 0.01298, val: 0.70211
Step:233, train: 0.01308, val: 0.70316
Step:234, train: 0.01286, val: 0.70421
Step:235, train: 0.01297, val: 0.70526
Step:236, train: 0.01294, val: 0.70631
Step:237, train: 0.01272, val: 0.70736
Step:238, train: 0.01250,

Step:424, train: 0.00987, val: 0.83774
Step:425, train: 0.01013, val: 0.83818
Step:426, train: 0.00993, val: 0.83862
Step:427, train: 0.00983, val: 0.83906
Step:428, train: 0.00990, val: 0.83949
Step:429, train: 0.01002, val: 0.83991
Step:430, train: 0.01018, val: 0.84034
Step:431, train: 0.00996, val: 0.84076
Step:432, train: 0.00985, val: 0.84119
Step:433, train: 0.00995, val: 0.84161
Step:434, train: 0.01000, val: 0.84203
Step:435, train: 0.00986, val: 0.84245
Step:436, train: 0.00995, val: 0.84288
Step:437, train: 0.00988, val: 0.84329
Step:438, train: 0.00967, val: 0.84371
Step:439, train: 0.00985, val: 0.84412
Step:440, train: 0.00987, val: 0.84453
Step:441, train: 0.01001, val: 0.84493
Step:442, train: 0.00966, val: 0.84533
Step:443, train: 0.01001, val: 0.84574
Step:444, train: 0.00981, val: 0.84614
Step:445, train: 0.00974, val: 0.84653
Step:446, train: 0.00992, val: 0.84693
Step:447, train: 0.00977, val: 0.84732
Step:448, train: 0.00984, val: 0.84771
Step:449, train: 0.00997,

Step:635, train: 0.00927, val: 0.89858
Step:636, train: 0.00938, val: 0.89877
Step:637, train: 0.00925, val: 0.89895
Step:638, train: 0.00929, val: 0.89914
Step:639, train: 0.00946, val: 0.89934
Step:640, train: 0.00936, val: 0.89953
Step:641, train: 0.00922, val: 0.89972
Step:642, train: 0.00917, val: 0.89991
Step:643, train: 0.00940, val: 0.90009
Step:644, train: 0.00912, val: 0.90028
Step:645, train: 0.00920, val: 0.90047
Step:646, train: 0.00923, val: 0.90066
Step:647, train: 0.00936, val: 0.90084
Step:648, train: 0.00918, val: 0.90102
Step:649, train: 0.00941, val: 0.90120
Step:650, train: 0.00915, val: 0.90138
Step:651, train: 0.00927, val: 0.90155
Step:652, train: 0.00943, val: 0.90172
Step:653, train: 0.00944, val: 0.90188
Step:654, train: 0.00920, val: 0.90205
Step:655, train: 0.00939, val: 0.90221
Step:656, train: 0.00923, val: 0.90238
Step:657, train: 0.00927, val: 0.90255
Step:658, train: 0.00923, val: 0.90272
Step:659, train: 0.00921, val: 0.90288
Step:660, train: 0.00939,

...

Step:1055, train: 0.00896, val: 0.93949
Step:1056, train: 0.00882, val: 0.93954
Step:1057, train: 0.00903, val: 0.93959
Step:1058, train: 0.00895, val: 0.93963
Step:1059, train: 0.00928, val: 0.93967
Step:1060, train: 0.00882, val: 0.93971
Step:1061, train: 0.00899, val: 0.93975
Step:1062, train: 0.00888, val: 0.93979
Step:1063, train: 0.00903, val: 0.93984
Step:1064, train: 0.00914, val: 0.93988
Step:1065, train: 0.00900, val: 0.93991
Step:1066, train: 0.00878, val: 0.93995
Step:1067, train: 0.00897, val: 0.93998
Step:1068, train: 0.00902, val: 0.94001
Step:1069, train: 0.00887, val: 0.94003
Step:1070, train: 0.00907, val: 0.94006
Step:1071, train: 0.00917, val: 0.94009
Step:1072, train: 0.00918, val: 0.94012
Step:1073, train: 0.00900, val: 0.94014
Step:1074, train: 0.00896, val: 0.94016
Step:1075, train: 0.00899, val: 0.94018
Step:1076, train: 0.00899, val: 0.94021
Step:1077, train: 0.00919, val: 0.94024
Step:1078, train: 0.00909, val: 0.94028
Step:1079, train: 0.00900, val: 0.94032


Step:1260, train: 0.00913, val: 0.94653
Step:1261, train: 0.00901, val: 0.94656
Step:1262, train: 0.00889, val: 0.94658
Step:1263, train: 0.00902, val: 0.94660
Step:1264, train: 0.00897, val: 0.94661
Step:1265, train: 0.00908, val: 0.94663
Step:1266, train: 0.00899, val: 0.94664
Step:1267, train: 0.00894, val: 0.94665
Step:1268, train: 0.00894, val: 0.94665
Step:1269, train: 0.00900, val: 0.94666
Step:1270, train: 0.00881, val: 0.94667
Step:1271, train: 0.00893, val: 0.94670
Step:1272, train: 0.00910, val: 0.94672
Step:1273, train: 0.00884, val: 0.94673
Step:1274, train: 0.00909, val: 0.94676
Step:1275, train: 0.00893, val: 0.94678
Step:1276, train: 0.00884, val: 0.94679
Step:1277, train: 0.00894, val: 0.94680
Step:1278, train: 0.00893, val: 0.94681
Step:1279, train: 0.00896, val: 0.94682
Step:1280, train: 0.00881, val: 0.94683
Step:1281, train: 0.00907, val: 0.94685
Step:1282, train: 0.00906, val: 0.94685
Step:1283, train: 0.00896, val: 0.94687
Step:1284, train: 0.00896, val: 0.94687


Step:1465, train: 0.00891, val: 0.94968
Step:1466, train: 0.00898, val: 0.94973
Step:1467, train: 0.00898, val: 0.94979
Step:1468, train: 0.00888, val: 0.94984
Step:1469, train: 0.00897, val: 0.94989
Step:1470, train: 0.00877, val: 0.94993
Step:1471, train: 0.00897, val: 0.94995
Step:1472, train: 0.00885, val: 0.94996
Step:1473, train: 0.00903, val: 0.94996
Step:1474, train: 0.00889, val: 0.94996
Step:1475, train: 0.00892, val: 0.94995
Step:1476, train: 0.00863, val: 0.94995
Step:1477, train: 0.00888, val: 0.94994
Step:1478, train: 0.00881, val: 0.94992
Step:1479, train: 0.00874, val: 0.94991
Step:1480, train: 0.00892, val: 0.94990
Step:1481, train: 0.00878, val: 0.94990
Step:1482, train: 0.00896, val: 0.94990
Step:1483, train: 0.00899, val: 0.94990
Step:1484, train: 0.00910, val: 0.94991
Step:1485, train: 0.00887, val: 0.94992
Step:1486, train: 0.00913, val: 0.94995
Step:1487, train: 0.00903, val: 0.94999
Step:1488, train: 0.00887, val: 0.95002
Step:1489, train: 0.00892, val: 0.95004


Step:1670, train: 0.00886, val: 0.95234
Step:1671, train: 0.00884, val: 0.95235
Step:1672, train: 0.00891, val: 0.95237
Step:1673, train: 0.00892, val: 0.95240
Step:1674, train: 0.00899, val: 0.95243
Step:1675, train: 0.00893, val: 0.95247
Step:1676, train: 0.00874, val: 0.95251
Step:1677, train: 0.00886, val: 0.95254
Step:1678, train: 0.00880, val: 0.95258
Step:1679, train: 0.00873, val: 0.95263
Step:1680, train: 0.00892, val: 0.95267
Step:1681, train: 0.00890, val: 0.95271
Step:1682, train: 0.00881, val: 0.95275
Step:1683, train: 0.00892, val: 0.95277
Step:1684, train: 0.00893, val: 0.95279
Step:1685, train: 0.00877, val: 0.95281
Step:1686, train: 0.00877, val: 0.95282
Step:1687, train: 0.00870, val: 0.95285
Step:1688, train: 0.00892, val: 0.95288
Step:1689, train: 0.00895, val: 0.95291
Step:1690, train: 0.00904, val: 0.95293
Step:1691, train: 0.00888, val: 0.95295
Step:1692, train: 0.00901, val: 0.95296
Step:1693, train: 0.00885, val: 0.95296
Step:1694, train: 0.00910, val: 0.95295


Step:1875, train: 0.00892, val: 0.95371
Step:1876, train: 0.00890, val: 0.95373
Step:1877, train: 0.00886, val: 0.95374
Step:1878, train: 0.00901, val: 0.95375
Step:1879, train: 0.00899, val: 0.95376
Step:1880, train: 0.00886, val: 0.95377
Step:1881, train: 0.00892, val: 0.95379
Step:1882, train: 0.00912, val: 0.95382
Step:1883, train: 0.00878, val: 0.95385
Step:1884, train: 0.00887, val: 0.95387
Step:1885, train: 0.00893, val: 0.95389
Step:1886, train: 0.00868, val: 0.95391
Step:1887, train: 0.00883, val: 0.95393
Step:1888, train: 0.00898, val: 0.95393
Step:1889, train: 0.00894, val: 0.95392
Step:1890, train: 0.00898, val: 0.95391
Step:1891, train: 0.00905, val: 0.95391
Step:1892, train: 0.00868, val: 0.95393
Step:1893, train: 0.00901, val: 0.95394
Step:1894, train: 0.00911, val: 0.95396
Step:1895, train: 0.00897, val: 0.95396
Step:1896, train: 0.00891, val: 0.95395
Step:1897, train: 0.00893, val: 0.95394
Step:1898, train: 0.00884, val: 0.95392
Step:1899, train: 0.00893, val: 0.95390


In [12]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

kraus_reg = KrausMap(
                       d = d, 
                       rank = d**2,
                       spam = spam_model,
                       )

model_reg = ModelQuantumMap(
                            q_map = kraus_reg,
                            loss = ProbabilityLoss(0.002),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            )

In [ ]:
model_reg.train(inputs = inputs_map,
                targets = targets_map,
                #inputs_val = inputs_map,
                targets_val = [kraus_target],
                num_iter = 1000,
                N = 500,
                verbose = True,
                )

  0%|          | 0/1000 [00:00<?, ?it/s]

Step:0, train: 0.21926, val: 0.09754
Step:1, train: 0.22615, val: 0.09946
Step:2, train: 0.22035, val: 0.10145
Step:3, train: 0.21991, val: 0.10354
Step:4, train: 0.21693, val: 0.10572
Step:5, train: 0.21308, val: 0.10797
Step:6, train: 0.21331, val: 0.11026
Step:7, train: 0.21696, val: 0.11259
Step:8, train: 0.20842, val: 0.11495
Step:9, train: 0.20675, val: 0.11737
Step:10, train: 0.20586, val: 0.11982
Step:11, train: 0.20503, val: 0.12233
Step:12, train: 0.19989, val: 0.12491
Step:13, train: 0.19843, val: 0.12755
Step:14, train: 0.19441, val: 0.13024
Step:15, train: 0.19602, val: 0.13296
Step:16, train: 0.18958, val: 0.13576
Step:17, train: 0.18414, val: 0.13861
Step:18, train: 0.19170, val: 0.14152
Step:19, train: 0.18202, val: 0.14448
Step:20, train: 0.18003, val: 0.14753
Step:21, train: 0.17710, val: 0.15062
Step:22, train: 0.17498, val: 0.15376
Step:23, train: 0.17228, val: 0.15697
Step:24, train: 0.17106, val: 0.16025
Step:25, train: 0.17051, val: 0.16359
Step:26, train: 0.1665

Step:213, train: 0.01829, val: 0.67041
Step:214, train: 0.01748, val: 0.67161
Step:215, train: 0.01766, val: 0.67280
Step:216, train: 0.01724, val: 0.67399
Step:217, train: 0.01751, val: 0.67517
Step:218, train: 0.01739, val: 0.67634
Step:219, train: 0.01749, val: 0.67750
Step:220, train: 0.01735, val: 0.67866
Step:221, train: 0.01756, val: 0.67980
Step:222, train: 0.01773, val: 0.68095
Step:223, train: 0.01708, val: 0.68209
Step:224, train: 0.01709, val: 0.68322
Step:225, train: 0.01673, val: 0.68435
Step:226, train: 0.01740, val: 0.68547
Step:227, train: 0.01713, val: 0.68660
Step:228, train: 0.01681, val: 0.68772
Step:229, train: 0.01692, val: 0.68884
Step:230, train: 0.01706, val: 0.68994
Step:231, train: 0.01709, val: 0.69104
Step:232, train: 0.01711, val: 0.69213
Step:233, train: 0.01702, val: 0.69322
Step:234, train: 0.01704, val: 0.69431
Step:235, train: 0.01668, val: 0.69539
Step:236, train: 0.01656, val: 0.69646
Step:237, train: 0.01683, val: 0.69753
Step:238, train: 0.01675,

Step:424, train: 0.01350, val: 0.83222
Step:425, train: 0.01345, val: 0.83266
Step:426, train: 0.01350, val: 0.83310
Step:427, train: 0.01357, val: 0.83354
Step:428, train: 0.01374, val: 0.83397
Step:429, train: 0.01353, val: 0.83440
Step:430, train: 0.01339, val: 0.83484
Step:431, train: 0.01360, val: 0.83527
Step:432, train: 0.01360, val: 0.83571
Step:433, train: 0.01338, val: 0.83615
Step:434, train: 0.01361, val: 0.83659
Step:435, train: 0.01343, val: 0.83703
Step:436, train: 0.01317, val: 0.83748
Step:437, train: 0.01350, val: 0.83792
Step:438, train: 0.01350, val: 0.83835
Step:439, train: 0.01357, val: 0.83878
Step:440, train: 0.01315, val: 0.83922
Step:441, train: 0.01358, val: 0.83965
Step:442, train: 0.01330, val: 0.84007
Step:443, train: 0.01334, val: 0.84050
Step:444, train: 0.01348, val: 0.84092
Step:445, train: 0.01338, val: 0.84135
Step:446, train: 0.01346, val: 0.84177
Step:447, train: 0.01358, val: 0.84220
Step:448, train: 0.01348, val: 0.84263
Step:449, train: 0.01335,

Step:635, train: 0.01267, val: 0.89804
Step:636, train: 0.01273, val: 0.89824
Step:637, train: 0.01287, val: 0.89844
Step:638, train: 0.01277, val: 0.89864
Step:639, train: 0.01264, val: 0.89884
Step:640, train: 0.01258, val: 0.89904
Step:641, train: 0.01281, val: 0.89925
Step:642, train: 0.01251, val: 0.89946
Step:643, train: 0.01265, val: 0.89967
Step:644, train: 0.01264, val: 0.89988
Step:645, train: 0.01275, val: 0.90008
Step:646, train: 0.01257, val: 0.90029
Step:647, train: 0.01281, val: 0.90049
Step:648, train: 0.01258, val: 0.90069
Step:649, train: 0.01272, val: 0.90089
Step:650, train: 0.01283, val: 0.90108
Step:651, train: 0.01283, val: 0.90127
Step:652, train: 0.01263, val: 0.90146
Step:653, train: 0.01279, val: 0.90164
Step:654, train: 0.01262, val: 0.90183
Step:655, train: 0.01269, val: 0.90202
Step:656, train: 0.01266, val: 0.90221
Step:657, train: 0.01260, val: 0.90241
Step:658, train: 0.01283, val: 0.90260
Step:659, train: 0.01271, val: 0.90280
Step:660, train: 0.01263,

Step:846, train: 0.01228, val: 0.92926
Step:847, train: 0.01249, val: 0.92936
Step:848, train: 0.01242, val: 0.92945
Step:849, train: 0.01243, val: 0.92955
Step:850, train: 0.01219, val: 0.92965
Step:851, train: 0.01243, val: 0.92975
Step:852, train: 0.01251, val: 0.92984
Step:853, train: 0.01238, val: 0.92994
Step:854, train: 0.01244, val: 0.93004
Step:855, train: 0.01242, val: 0.93014
Step:856, train: 0.01240, val: 0.93024
Step:857, train: 0.01244, val: 0.93035
Step:858, train: 0.01248, val: 0.93045
Step:859, train: 0.01232, val: 0.93054
Step:860, train: 0.01250, val: 0.93064
Step:861, train: 0.01229, val: 0.93074
Step:862, train: 0.01248, val: 0.93084
Step:863, train: 0.01219, val: 0.93094
Step:864, train: 0.01217, val: 0.93104
Step:865, train: 0.01233, val: 0.93114
Step:866, train: 0.01236, val: 0.93123
Step:867, train: 0.01251, val: 0.93133
Step:868, train: 0.01256, val: 0.93143
Step:869, train: 0.01255, val: 0.93152
Step:870, train: 0.01245, val: 0.93162
Step:871, train: 0.01235,